# Building Models to rationalize SAR

So far, we have compiled a dataset containing binding data for various PKA binding molecules. In this second part of the practical, we will try to build simple models that can help us rationalize why and who the molecules bind to the protein. Importantly, we want to learn the Structure Activity Relationship from the model and try to propose new compounds given the ones we already know.

First, load the data from the previous notebook from the `.csv` save on disk.

In [ ]:
import pandas as pd
output_df = pd.read_csv("./binders.csv")

## Data Preparation

We will first generate features (i.e. the independant variables in our dataset). In the previous step, we generated features according to Ro5. However this time, want to generate lots of additional features and see if they contain any additional information that will be useful for us further down the road.

First things first, generate the features and try to understand them. Use `print(list(rich_descriptors.keys()))` to get the full list of descriptors.

1.) In the cell below, we are also filtering the data to exclude features that have zero variance. Explain why.

In [ ]:
import helpers

rich_descriptors = output_df["smiles"].apply(helpers.calculate_all_features)
### Filter out features with zero variance
variance = rich_descriptors.var()
columns  = list()
for c,v in variance.items():
    if v > 0.:
        columns.append(c)
rich_descriptors = rich_descriptors[columns]
data_rich_descriptors = output_df[["molecule_chembl_id", "pIC50", "IC50"]].join(rich_descriptors)

### These are the Ro5 features
features_Ro5 = ["ExactMolWt", "NumHAcceptors", "NumHDonors", "MolLogP", "NumRotatableBonds"]
### These are the advanced features
features_advanced = features_Ro5 + [
    "MaxAbsEStateIndex", "MaxEStateIndex", "MinAbsEStateIndex", "MinEStateIndex", 
    "qed", "SPS", "MaxPartialCharge", "MinPartialCharge", "MaxAbsPartialCharge", 
    "MinAbsPartialCharge", "SlogP_VSA1", "SlogP_VSA2", "SlogP_VSA3", "SlogP_VSA4", 
    "SlogP_VSA5", "SlogP_VSA6", "SlogP_VSA7", "SlogP_VSA8", "SlogP_VSA10", "SlogP_VSA11", 
    "SlogP_VSA12", "PEOE_VSA1", "PEOE_VSA2", "PEOE_VSA3", "PEOE_VSA4", "PEOE_VSA5", 
    "PEOE_VSA6", "PEOE_VSA7", "PEOE_VSA8", "PEOE_VSA9", "PEOE_VSA10", "PEOE_VSA11", 
    "PEOE_VSA12", "PEOE_VSA13", "PEOE_VSA14", "SMR_VSA1", "SMR_VSA2", "SMR_VSA3", 
    "SMR_VSA4", "SMR_VSA5", "SMR_VSA6", "SMR_VSA7", "SMR_VSA9", "TPSA", "HallKierAlpha", 
    "Kappa1", "Kappa2", "Kappa3", "BertzCT", "AvgIpc"
]

data_rich_descriptors.head()

## Advanced descriptors

In the following we will list advanced descriptors that are stored in the above dataframe.

### Electrotopological state

The electropological state (E-state) encodes the topology and electronic environment of each atom in a molecule. In the above dataframe, the features corresponding to the E-State are `MaxAbsEStateIndex`,  `MaxEStateIndex`, `MinAbsEStateIndex` and `MinEStateIndex`. Reference *J. Chem. Inf. Comput. Sci. 1991, 31, 76-82*

### Drug-likliness

The QED descriptor describes the drug-likeliness of a molecule. The namer of the descriptor in the above dataframe is `qed`. Reference: *Bickerton, G.R.; Paolini, G.V.; Besnard, J.; Muresan, S.; Hopkins, A.L. (2012) ‘Quantifying the chemical beauty of drugs’, Nature Chemistry, 4, 90-98*

### Spatial complexity

The Spatial complexity is described by the `SPS` descriptor. Reference *Krzyzanowski, A.; Pahl, A.; Grigalunas, M.; Waldmann, H. Spacial Score─A Comprehensive Topological Indicator for Small-Molecule Complexity. J. Med. Chem. 2023*

### Partial Charges

The Gasteiger partial charge of each atom in a molecule is physical parameter describing its static electric environment in a molecule. The corresponding decsriptors are `MaxPartialCharge`, `MinPartialCharge`, `MaxAbsPartialCharge` and `MinAbsPartialCharge`.

### Lipophilicity

The Lipophilicity of a molecule is described by a whole set of descriptors. In the above dataframe they are `SlogP_VSAX` (X=1-12). Reference: *Paul Labute, Journal of Molecular Graphics and Modelling 18, 464-477, 2000*. This is an interesting blog post: https://greglandrum.github.io/rdkit-blog/posts/2023-04-17-what-are-the-vsa-descriptors.html 

### Electrostatic interactions

Electrostatic interactions descriptor. `PEOE_VSAX` (X=1-14). Same reference as above.

### Molecular Polarazibility

Molecular Polarazibility descriptor. `SMR_VSAX` (X=1-9). Same reference as above.

### Polar Surface Area

The total polar surface area is described by the `TPSA` descriptor. This descriptor describes the surface area in a given molecule that is attributed to polar atoms.

### Topological Indices 1

The topological indices developed by Hall and Kier describe the topology of the underlying molecule. In the above dataframe they are `HallKierAlpha`, `Kappa1`, `Kappa2` and `Kappa3`. Reference *Lowell H. Hall and Lemont B. Kier, Reviews in Computational Chemistry, Volume 2, 1991*

### Topological Indices 2

The `BertzCT` and `AvgIpc` descriptors quantify the topological properties of the underlying molecular graphs. The `BertzCT` quantifies molecule complexity of molecules. The `AvgIpc` qunatifies the information content contained in the adjacency matrix of the molecular graph. References *From S. H. Bertz, J. Am. Chem. Soc., vol 103, 3599-3601 (1981)* and *D. Bonchev & N. Trinajstic, J. Chem. Phys. vol 67, 4517-4533 (1977)*

## Build a linear Multiple Least Squares Model

### Test set size and model uncertainty

Now, let us build a linear model using the features extraced in the previous step. Luckily we don't have to do that all by hand, but we can use existing model building architectures contained in the `scikit-learn` python package.
We will first have to decide on the size of training set. In this case, we will use 25% of the data for testing. To evaluate the quality of our model, we will compute the coefficient of determination (R2) and the mean unsigned error (MUE). In addition we will report the 95% confidence interval obtained from `n_evaluations=600` repeated fits of our model. During each repitition a random split into test/training set was carried out. The results with CI are reported as **MEAN [LOWER_CI UPPER_CI]**.

1.) What exactly is the different between test set and training set? Why do we need these two different sets of data?

2.) How does the model perform with increasing/decreasing training/test set size?

3.) What is a confidence interval and why is it useful? Tip: Look up the Central Limit Theorem.

4.) How could you test whether the set of `R2` and `MUE` are actually Gaussian distributed?

5.) Look at the linear coefficients. What do they tell you about the importance of the different features?

6.) Repeat the analysis with the Ro5 features `features_Ro5`. How do you explain the decrease in performance? Note, only change the value of the `INPUT_FEATURES`.

In [ ]:
from sklearn import linear_model
import numpy as np

INPUT_FEATURES = features_advanced

results, _ = helpers.fit_model(
    model = linear_model.LinearRegression(), # model
    n_evaluations = 1000, # number of attempts to generate a model
    test_size = 0.10, # size of the test set
    dataset = data_rich_descriptors, # full dataset
    x_name = INPUT_FEATURES, # names of the descriptors (independant variable)
    y_name = "pIC50", #name of dependent variable
    properties = ["coef_"], # names of the properties to extract from model
)

R2_test       = results['R2_test'].mean()
MUE_test      = results['MUE_test'].mean()
CI95_R2_test  = helpers.get_CI(results['R2_test'])
CI95_MUE_test = helpers.get_CI(results['MUE_test'])

print(
    f'Performance Statistics:', '\n'
    f'-----------------------', '\n'
    f'R2 test set : {R2_test:4.2f} [{CI95_R2_test[0]:4.2f} {CI95_R2_test[1]:4.2f}]', '\n'
    f'MUE test set: {MUE_test:4.2f} [{CI95_MUE_test[0]:4.2f} {CI95_MUE_test[1]:4.2f}]', '\n'
)
print(
    'Linear coefficients:','\n'
    '--------------------'
)
coef_ = np.array(results['coef_'])
sorted_idxs = np.argsort(np.abs(coef_.mean(axis=0)))[::-1]
for idx in sorted_idxs:
    p = INPUT_FEATURES[idx]
    v = coef_[:,idx].mean()
    CI95 = helpers.get_CI(coef_[:,idx])
    print(
        f'{p:20s} : {v:4.6f} [{CI95[0]:4.6f} {CI95[1]:4.6f}]'
    )

### Model convergence

Run the cell below to plot the average `R2` and `MUE` over the course of the fitting.

1.) Explain the strong fluctuations in the beginning the fitting and why they become less later on.



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axs = plt.subplots(nrows=1, ncols=2)

data   = results["MUE_test"]
N      = data.size
runavg = np.cumsum(data)/np.arange(1,N+1)
axs[0].plot(runavg)
axs[0].set_title(r"$MUE$ test set")
axs[0].set_ylabel("MUE")
axs[0].set_xlabel("Iteration")

data   = results["R2_test"]
N      = data.size
runavg = np.cumsum(data)/np.arange(1,N+1)
axs[1].plot(runavg)
axs[1].set_title(r"$R^2$ test set")
axs[1].set_ylabel(r"$R^2$")
axs[1].set_xlabel("Iteration")

fig.tight_layout()
fig.show()

### Use other features and models

In the next step, we will repeat our experiments from above but this time we will use a Random Forest regressor.

1.) Read up on the random forest regressor and understand how it works. How is this regressor related to decision trees?

2.) Repeat the model fitting with a random forest model. Below, the value of the model parameter of the function  `helpers.fit_model` was set to `ensemble.RandomForestRegressor()`.

3.) Random Forests are able to rank the input features based on how much they contribute to the prediction. Look at the feature importance value reported below and explain which features seem to be more important than others. Also relate your findings to the analysis of the linear coefficients in the Linear Least Square Model above. 

4.) Repeat the analysis with the Ro5 features `features_Ro5`. Explain the observed difference.

Note: The cell below will save the best model that was found over `n_evaluations`. The best model will be stored in the variable `reg_model`. Keep this in mind when running the last cell of this notebook.

In [ ]:
from sklearn import dummy
from sklearn import linear_model
from sklearn import ensemble

INPUT_FEATURES = features_advanced

results, reg_model = helpers.fit_model(
    model = ensemble.RandomForestRegressor(), # model
    n_evaluations = 100, # number of attempts to generate a model
    test_size = 0.10, # size of the test set
    dataset = data_rich_descriptors, # full dataset
    x_name = INPUT_FEATURES, # names of the descriptors (independant variable)
    y_name = "pIC50", #name of dependent variable
    properties = ["feature_importances_"], # names of the properties to extract from model
)

R2_test  = results['R2_test'].mean()
MUE_test = results['MUE_test'].mean()
CI95_R2_test = helpers.get_CI(results['R2_test'])
CI95_MUE_test = helpers.get_CI(results['MUE_test'])

print(
    f'Performance Statistics:', '\n'
    f'-----------------------', '\n'
    f'R2 test set : {R2_test:4.2f} [{CI95_R2_test[0]:4.2f} {CI95_R2_test[1]:4.2f}]', '\n'
    f'MUE test set: {MUE_test:4.2f} [{CI95_MUE_test[0]:4.2f} {CI95_MUE_test[1]:4.2f}]', '\n'
)

print(
    'Feature importance:','\n'
    '-------------------'
)
coef_ = np.array(results['feature_importances_'])
sorted_idxs = np.argsort(coef_.mean(axis=0))[::-1]
for idx in sorted_idxs:
    p = INPUT_FEATURES[idx]
    v = coef_[:,idx].mean()
    CI95 = helpers.get_CI(coef_[:,idx])
    print(
        f'{p:20s} : {v:4.6f} [{CI95[0]:4.6f} {CI95[1]:4.6f}]'
    )

## Testing new molecules

Imagine that now after building a nice predictive model, you are asked by a Medicinal Chemist to provide a list of four molecules that will bind to our target protein. They provide you with an initial list of eight molecules and you are asked to tell them the ones that are most likely to bind.

In [ ]:
from rdkit.Chem import Descriptors, Draw, PandasTools
import pandas as pd

test_mols = [
    "O=S(C2=CC=CC1=CN=CC=C12)(N3CCNC[C@@H]3C)=O",
    "O=S(C2=CC=CC1=CN=CC=C12)(N3CCCNCC3)=O",
    "O=S(C2=CC=CC1=CN=CC=C12)(N(CCC)CCN)=O",
    "O=S(C2=CC=CC1=CN=CC=C12)(NCCNCCC)=O",
    "O=S(C2=CC=CC1=CN=CC=C12)(NCCNC(CCC)CC)=O",
    "O=S(C2=CC=CC1=CN=CC=C12)(NC(CCSCNO)CNCCC)=O",
    "O=C(C2=CC=CC1=CN=CC=C12)(NCCNCCC)",
    "O=S(C2=CC=CC1=CC=CC=C12)(N3CCCNCC3)=O"
]

molecules = pd.DataFrame(
    {"smiles" : pd.Series(test_mols,                          
                         )
    }
)
PandasTools.RenderImagesInAllDataFrames(images=True)
PandasTools.AddMoleculeColumnToFrame(molecules, "smiles", includeFingerprints=True)
molecules

**Which molecules will you select based on your model?**

In [ ]:
rich_descriptors = molecules["smiles"].apply(helpers.calculate_all_features)
reg_model.predict(rich_descriptors[INPUT_FEATURES])

## Conclusion

Great, you've worked your way all through the notebook. Make sure that you have a good answer for each of the questions above as they will guide you through the report.